# 1. 使用FUNCTION_CALL模式

In [6]:
import os
import dotenv
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.chains.question_answering.map_reduce_prompt import system_template
from langchain_community.tools import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.tools import Tool

# 加载环境
dotenv.load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY1")
os.environ["OPENAI_BASE_URL"] = os.getenv("OPENAI_BASE_URL")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

# 获取tavily搜索工具的实例
search = TavilySearchResults(max_results=3)
# 获取对应工具的实例
search_tool = Tool(
	name="tavily_search",
	func=search.run,
	description="一个可以用来搜索互联网信息的工具。输入是一个搜索查询，输出是搜索结果的摘要。",
	return_direct=True
)
# 获取大模型
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
# 提供提示词模板（以ChatPromptTemplate为例）
prompt_template = ChatPromptTemplate.from_messages(
	[
		("system", "你是一个有帮助的助手。可以回答用户的问题。如果你不知道答案，请使用tavily_search工具来搜索互联网信息。"),
		("human", "{input}"),
		("placeholder", "{agent_scratchpad}")
	]
)
# 获取Agent的实例：create_tool_calling_agent()
agent = create_tool_calling_agent(
	llm=llm,
	tools=[search_tool],
	prompt=prompt_template
)
# 获取AgentExecutor的实例
agent_executor = AgentExecutor(
	agent=agent,
	tools=[search_tool],
	verbose=True
)
# 通过AgentExecutor调用invoke()方法,并得到响应
result = agent_executor.invoke({"input": "2025年10月的最新AI技术趋势是什么？"})
# 处理响应
print(result)



> Entering new AgentExecutor chain...

Invoking: `tavily_search` with `2025 AI technology trends October`


[{'title': 'AI Product Launches October 2025 & Tools for Business Automation', 'url': 'https://www.aiapps.com/blog/ai-news-breakthroughs-launches-trends/', 'content': "The trends set in motion during October 2025 are poised to influence the AI industry for years to come. This period of rapid advancement is paving the way for broader AI adoption. With increasingly sophisticated technologies - from generative models to synthetic data solutions - and discovery platforms like AI Apps, businesses of all sizes are better equipped to harness AI for growth and innovation.\n\nFAQs [...] October 2025 marked a pivotal moment for artificial intelligence, redefining how automation, creativity, and business processes are approached. These advancements not only pushed AI's potential further but also reshaped strategies across industries. Generative and agent-based AI models raised the bar for

# 2. 使用ReAct模式

举例1：使用PromptTemplate

In [8]:
import os
import dotenv
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools import TavilySearchResults
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.tools import Tool

# 加载环境
dotenv.load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY1")
os.environ["OPENAI_BASE_URL"] = os.getenv("OPENAI_BASE_URL")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

# 获取tavily搜索工具的实例
search = TavilySearchResults(max_results=3)
# 获取对应工具的实例
search_tool = Tool(
	name="tavily_search",
	func=search.run,
	description="一个可以用来搜索互联网信息的工具。输入是一个搜索查询，输出是搜索结果的摘要。",
	return_direct=True
)
# 获取大模型
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
# 提供提示词模板（以PromptTemplate为例）
template = """
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
"""
prompt_template = PromptTemplate.from_template(
	template=template
)
# 获取Agent的实例：create_react_agent()
agent = create_react_agent(
	llm=llm,
	tools=[search_tool],
	prompt=prompt_template
)
# 获取AgentExecutor的实例
agent_executor = AgentExecutor(
	agent=agent,
	tools=[search_tool],
	verbose=True
)
# 通过AgentExecutor调用invoke()方法,并得到响应
result = agent_executor.invoke({"input": "未来什么行业比较火？"})
# 处理响应
print(result)



> Entering new AgentExecutor chain...
为了回答这个问题，我需要查找关于未来热门行业的最新信息和趋势。  
Action: tavily_search  
Action Input: "未来热门行业 2024"  [{'title': '2024年全球并购趋势展望 - 普华永道', 'url': 'https://www.pwccn.com/zh/services/deals-m-and-a/publications/global-ma-industry-trends-2024.html', 'content': 'Menu 行业 行业 Menu 行业 金融服务 Menu 行业 资产和财富管理 Menu 行业 银行及资本市场 Menu 行业 保险 Menu 行业 私募股权基金 Menu 行业 科技、媒体和通信 Menu 行业 医药医疗及大健康 Menu 行业 消费市场 Menu 行业 汽车 Menu 行业 工业产品 Menu 行业 工程建筑 Menu 行业 防务及航空航天 Menu 行业 运输与物流 Menu 行业 房地产 Menu 行业 政府及公共服务 Menu 行业 食品行业诚信服务 Menu 行业 航空 Menu 行业 能源、公用事业及采矿 Menu 行业 基础设施与大型项目投融资服务 Menu 重要议题 重要议题 Menu 重要议题 全球跨境服务 Menu 重要议题 网络安全和隐私保护 Menu 重要议题 科创与民营企业 Menu 重要议题 环境、社会及治理 (ESG) Menu 重要议题 生成式人工智能 (GenAI) Menu 重要议题 粤港澳大湾区 Menu 重要议题 新基建 Menu 重要议题 新时代。新技能。 Menu 重要议题 掌控风险平台 Menu 重要议题 科创板 Menu 重要议题 国企改革 Menu 重要议题 人才和组织 Menu 研究与洞察 研究与洞察 Menu Copy link Link copied to clipboard 2023年全球能源、公用事业和资源的并购交易数量及交易金额较上年分别增长了1%和26%。尽管许多其他行业的交易有所下降，然而受到能源转型的重要性驱动，能源、公用事业和资源领域持续吸引投资者进入，全球并购交易活动依然活跃。 全球制药与生命科学和医疗服务业并购在2023年仍

上述方式可以改写成远程获取提示词模版

In [11]:
import os
import dotenv
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools import TavilySearchResults
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.tools import Tool
from langchain import hub

# 加载环境
dotenv.load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY1")
os.environ["OPENAI_BASE_URL"] = os.getenv("OPENAI_BASE_URL")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

# 获取tavily搜索工具的实例
search = TavilySearchResults(max_results=3)
# 获取对应工具的实例
search_tool = Tool(
	name="tavily_search",
	func=search.run,
	description="一个可以用来搜索互联网信息的工具。输入是一个搜索查询，输出是搜索结果的摘要。",
	return_direct=True
)
# 获取大模型
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
# 提供提示词模板（使用langchain hub中官方ReAct提示词模板）
prompt_template = hub.pull("hwchase17/react")
# 获取Agent的实例：create_react_agent()
agent = create_react_agent(
	llm=llm,
	tools=[search_tool],
	prompt=prompt_template
)
# 获取AgentExecutor的实例
agent_executor = AgentExecutor(
	agent=agent,
	tools=[search_tool],
	verbose=True
)
# 通过AgentExecutor调用invoke()方法,并得到响应
result = agent_executor.invoke({"input": "未来什么行业比较火？"})
# 处理响应
print(result)



> Entering new AgentExecutor chain...
我需要了解当前和未来的行业趋势，以确定哪些行业可能会变得热门。为了获取最新的信息，我将进行搜索。  
Action: tavily_search  
Action Input: '未来热门行业 2024'  [{'title': '目前什么行业最有发展前景？2024年热门行业大盘点_领域 - 搜狐', 'url': 'https://www.sohu.com/a/809247619_100054224', 'content': '1. 制造业：通过应用数字技术和智能制造改变生产模式，提高生产柔性和效率。\n2. 金融行业：IT服务部门将数据转变为业务价值，与业务部门合作，共同挖掘和利用数据的潜在价值。\n3. 医疗健康：涵盖药物发现、辅助诊断、个性化治疗等多个方面，加快药物开发和疾病发现。\n4. 教育领域：数字化战略对教师队伍提出了新的要求，以数字化赋能高质量教师队伍建设。\n5. 企业服务行业：包括金融、电子商务和软件信息等行业，为其他行业提供数字化转型支持。\n\n# 区块链技术\n\n区块链和云计算技术的普及与应用将带来新的就业机会，特别是在金融、保险和网络安全等领域。\n\n应用案例：\n\n1. 金融领域：华夏银行与益邦控股、腾讯集团合作的“星云贝链”，展示了区块链技术在供应链金融中的应用。\n2. 保险领域：信美人寿的爱心救助账户，利用区块链技术实现相互保险。\n3. 网络安全领域：去中心化的身份管理系统，增强网络身份的安全性；不可变的审计跟踪，提供可靠的审计记录。 [...] 2025年10月10日 IT频道最新文章\nIT频道最新文章\n\n#### IT见闻录\n\n:   0文章\n:   4.6万总阅读\n\n# 目前什么行业最有发展前景？2024年热门行业大盘点\n\n2024-09-15 21:17 \n\n发布于：北京市\n\n在27岁的年纪决定转行，尤其是零基础的情况下，选择一个有良好就业前景的行业是至关重要的。未来五年内，有几个领域将迎来显著的增长和发展，提供了丰富的就业机会。以下是一些推荐的行业和职业选择：\n\n# 人工智能与机器学习\n\n根据世界经济论坛的报告，到2027年，人工智能和机器学习专家的工作岗位预计将增长40%，相当于约100万个新工

举例2：使用ChatPromptTemplate

In [19]:
import os
import dotenv
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain.tools import Tool

# 加载环境
dotenv.load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY1")
os.environ["OPENAI_BASE_URL"] = os.getenv("OPENAI_BASE_URL")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

# 获取tavily搜索工具的实例
search = TavilySearchResults(max_results=3)
# 获取对应工具的实例
search_tool = Tool(
	name="tavily_search",
	func=search.run,
	description="一个可以用来搜索互联网信息的工具。输入是一个搜索查询，输出是搜索结果的摘要。",
	return_direct=True
)
# 获取大模型
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
# 提供提示词模板（以ChatPromptTemplate为例）
system_template = """
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!
"""
prompt_template = ChatPromptTemplate.from_messages([
	("system", system_template),
	("human", "{input}"),
	("system", "{agent_scratchpad}")
])
# 获取Agent的实例：create_react_agent()
agent = create_react_agent(
	llm=llm,
	tools=[search_tool],
	prompt=prompt_template
)
# 获取AgentExecutor的实例
agent_executor = AgentExecutor(
	agent=agent,
	tools=[search_tool],
	verbose=True
)
# 通过AgentExecutor调用invoke()方法,并得到响应
result = agent_executor.invoke({"input": "凤姐还在美国吗？"})
# 处理响应
print(result)



> Entering new AgentExecutor chain...
Question: 凤姐还在美国吗？
Thought: 我需要查找关于凤姐目前居住地的信息。
Action: tavily_search
Action Input: "凤姐 还在美国吗 2023"[{'title': '39岁凤姐近况曝光！住15刀民宿发福至160斤 - 文学城', 'url': 'https://www.wenxuecity.com/news/2025/03/21/ent-270827.html', 'content': '。这些夸张荒诞的言论，使她迅速成为网络审丑文化的典型代表，一时间声名大噪。然而，成名的背后也伴随着巨大的代价。在汶川地震期间，凤姐因发表不当言论，遭到全网封杀。随后，她喊出 “宁死不回中国” 的口号，毅然决然地远赴美国，追逐心中的 “美国梦”。初到美国的凤姐，日子过得并不顺遂。由于语言不通、文化存在差异，她只能辗转于美甲店、超市等底层工作岗位，拿着微薄的时薪，艰难维持生计。据说她当时的时薪仅有 8 美元，居住环境更是恶劣，只能在潮湿的地下室中栖身。即便后来在 2023 年考取了美甲师执照，月收入有所提升，达到了约 2000 美元，但这与网传的 “月入 6000 美元” 相差甚远。此次曝光的 15 美元民宿，房间设施极为简陋，只有一张行军床，洗漱还需要使用公共卫生间，与她早年宣称的 “住曼哈顿高级公寓” 相比，简直是天壤之别。如今的凤姐，身材发福，体重飙升至 160 斤，整个人看起来疲惫又无奈，曾经那个在网络上风光无限的 “网红鼻祖”，如今已然成为美国底层移民的一个缩影。值得一提的是，在生活如此艰难的情况下，凤姐并没有放弃对知识的追求。2022 [...] 。这些夸张荒诞的言论，使她迅速成为网络审丑文化的典型代表，一时间声名大噪。然而，成名的背后也伴随着巨大的代价。在汶川地震期间，凤姐因发表不当言论，遭到全网封杀。随后，她喊出 “宁死不回中国” 的口号，毅然决然地远赴美国，追逐心中的 “美国梦”。初到美国的凤姐，日子过得并不顺遂。由于语言不通、文化存在差异，她只能辗转于美甲店、超市等底层工作岗位，拿着微薄的时薪，艰难维持生计。据说她当时的时薪仅有 8 美元，居住环境更是恶劣，只能在潮湿的地下室中栖身。即便后来在 2023 年考取了美甲师执照，月收入有所

# 3. 小结

1.传统方式，相较于通用方式来讲，不需要提示词模板。

2.对于通用方式来讲
- FUNCTION_CALL模式：在创建agent时，推荐使用ChatPromptTemplate
- ReAct模式：需要提供提示词模板，可以使用PromptTemplate或ChatPromptTemplate，相较而言推荐使用PromptTemplate。